<a href="https://colab.research.google.com/github/liu-kevin1/ComplementaryClothes/blob/master/CS_Project_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Features to get from audio file:

'length'

'chroma_stft_mean'

'chroma_stft_var'

'rms_mean'

'rms_var'

'spectral_centroid_mean'

'spectral_centroid_var'

'spectral_bandwidth_mean'

'spectral_bandwidth_var'

'rolloff_mean'

'rolloff_var'

'zero_crossing_rate_mean'

'zero_crossing_rate_var'

'harmony_mean'

'harmony_var'

'perceptr_mean'

'perceptr_var'

'tempo'

'mfcc1_mean'

'mfcc1_var'

'mfcc2_mean'

'mfcc2_var'

'mfcc3_mean'

'mfcc3_var'

'mfcc4_mean'

'mfcc4_var'

'mfcc5_mean'

'mfcc5_var'

'mfcc6_mean'

'mfcc6_var'

'mfcc7_mean'

'mfcc7_var'

'mfcc8_mean'

'mfcc8_var'

'mfcc9_mean'

'mfcc9_var'

'mfcc10_mean'

'mfcc10_var'

'mfcc11_mean'

'mfcc11_var'

'mfcc12_mean'

'mfcc12_var'

'mfcc13_mean'

'mfcc13_var'

'mfcc14_mean'

'mfcc14_var'

'mfcc15_mean'

'mfcc15_var'

'mfcc16_mean'

'mfcc16_var'

'mfcc17_mean'

'mfcc17_var'

'mfcc18_mean'

'mfcc18_var'

'mfcc19_mean'

'mfcc19_var'

'mfcc20_mean'

'mfcc20_var'



In [26]:
# Imports
import numpy as np
import pandas as pd
import librosa

# File Imports
from Song import Song

from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

df = pd.read_csv('features_3_sec.csv')
df2 = pd.read_csv('features_30_sec.csv')

labels = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

df = df.drop('perceptr_var', axis=1).drop('perceptr_mean', axis=1)
df2 = df2.drop('perceptr_var', axis=1).drop('perceptr_mean', axis=1)

y = df['label']
X = df.drop('label', axis = 1).drop('filename', axis=1).drop('length', axis=1)

y2 = df2['label']
X2 = df2.drop('label', axis = 1).drop('filename', axis=1).drop('length', axis=1)

# Scale the features
cols = X.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)

X = pd.DataFrame(np_scaled, columns = cols)
# print(X)
cols = X2.columns
#min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.transform(X2)
X2 = pd.DataFrame(np_scaled, columns = cols)
# print(X2)
# sns.heatmap(df.corr(), square=True, cmap='RdYlGn')



      chroma_stft_mean  chroma_stft_var  ...  mfcc20_mean  mfcc20_var
0             0.355399         0.716757  ...     0.506746    0.047781
1             0.367322         0.670347  ...     0.593029    0.065548
2             0.373159         0.728067  ...     0.546264    0.036062
3             0.399349         0.677066  ...     0.562204    0.034873
4             0.355668         0.689113  ...     0.517913    0.031713
...                ...              ...  ...          ...         ...
9985          0.376757         0.617031  ...     0.536263    0.042502
9986          0.413243         0.637013  ...     0.516365    0.020232
9987          0.374196         0.697548  ...     0.505937    0.045380
9988          0.436537         0.657745  ...     0.519900    0.013740
9989          0.408151         0.676143  ...     0.461372    0.034548

[9990 rows x 55 columns]
     chroma_stft_mean  chroma_stft_var  ...  mfcc20_mean  mfcc20_var
0            0.378254         0.695059  ...     0.527709    0.051

In [24]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.1 , random_state = 42, stratify = y)
X2_train, X2_test, y2_train, y2_test = train_test_split (X2, y2, test_size = 0.1 , random_state = 42, stratify = y2)

neighbors = range(5, 20)
test_accuracy = np.empty(len(neighbors))
highest_score = 0
highest_i = 0
best_knn = 0
confus_mat = 0

for i, k in enumerate(neighbors):
    # Setup a k-NN Classifier with k neighbors: knn
    knn = KNeighborsClassifier(n_neighbors = k)

    # Fit the classifier to the training data
    knn.fit(X, y)

    #Compute accuracy on the testing set
    test_accuracy[i] = knn.score(X2_test, y2_test)
    if test_accuracy[i] > highest_score:
      highest_score = test_accuracy[i]
      highest_i = k
      best_knn = knn
      
      preds = knn.predict(X_test)
      confus_mat = confusion_matrix(y_test, preds) 
    
# plt.figure(figsize = (10, 5))
# sns.heatmap(confus_mat)

print("Highest Score: %.2f; i: %d" % (highest_score, highest_i))

Highest Score: 0.98; i: 6


In [25]:
knn = best_knn

# Find:
# 'length'
fname = 'blues.00000.wav'

def get_input_from_file(fname):
  x, fs = librosa.load(fname)

  with contextlib.closing(wave.open(fname,'r')) as f:
      frames = f.getnframes()
      rate = f.getframerate()
      duration = frames / float(rate)

  # 'chroma_stft'
  chroma_stft = librosa.feature.chroma_stft(x, fs)
  chroma_stft_mean = np.mean(chroma_stft)
  chroma_stft_var = np.var(chroma_stft)

  # 'rms'
  rms = librosa.feature.rms(x)
  rms_mean = np.mean(rms)
  rms_var = np.var(rms)

  # 'spectral_centroid'
  spectral_centroid = librosa.feature.spectral_centroid(x, fs)
  spectral_centroid_mean = np.mean(spectral_centroid)
  spectral_centroid_var = np.var(spectral_centroid)

  # 'spectral_bandwidth'
  spectral_bandwidth = librosa.feature.spectral_bandwidth(x, fs)
  spectral_bandwidth_mean = np.mean(spectral_bandwidth)
  spectral_bandwidth_var = np.var(spectral_bandwidth)

  # 'rolloff'
  rolloff = librosa.feature.spectral_rolloff(x, fs)
  rolloff_mean = np.mean(rolloff)
  rolloff_var = np.var(rolloff)

  # 'zero_crossing_rate'
  zero_crossing_rate = librosa.feature.zero_crossing_rate(x)
  zero_crossing_rate_mean = np.mean(zero_crossing_rate)
  zero_crossing_rate_var = np.var(zero_crossing_rate)

  # 'harmony'
  # harmony = librosa.feature.tonnetz(x, fs)
  harmony = librosa.effects.harmonic(x)
  harmony_mean = np.mean(harmony)
  harmony_var = np.var(harmony)

  # Ignoring the perceptr input because I can't figure out what feature it is/how to get it
  # spect = librosa.feature.melspectrogram(x, fs)
  # freqs = librosa.fft_frequencies(fs)
  # # 'perceptr'
  # perceptr = librosa.perceptual_weighting(spect, freqs)
  # perceptr_mean = np.mean(perceptr)
  # perceptr_var = np.var(perceptr)

  # 'tempo'
  tempo = librosa.beat.tempo(x, fs)[0]

  # Find mfccs:
  # mean
  # var(iance)
  mfccs = librosa.feature.mfcc(x, sr=fs)

  inp = np.array([chroma_stft_mean, chroma_stft_var, 
                  rms_mean, rms_var, 
                  spectral_centroid_mean, spectral_centroid_var, 
                  spectral_bandwidth_mean, spectral_bandwidth_var, 
                  rolloff_mean, rolloff_var, 
                  zero_crossing_rate_mean, zero_crossing_rate_var, 
                  harmony_mean, harmony_var, 
                  tempo, 
                  np.mean(mfccs[0]), np.var(mfccs[0]),
                  np.mean(mfccs[1]), np.var(mfccs[1]),
                  np.mean(mfccs[2]), np.var(mfccs[2]),
                  np.mean(mfccs[3]), np.var(mfccs[3]),
                  np.mean(mfccs[4]), np.var(mfccs[4]),
                  np.mean(mfccs[5]), np.var(mfccs[5]),
                  np.mean(mfccs[6]), np.var(mfccs[6]),
                  np.mean(mfccs[7]), np.var(mfccs[7]),
                  np.mean(mfccs[8]), np.var(mfccs[8]),
                  np.mean(mfccs[9]), np.var(mfccs[9]),
                  np.mean(mfccs[10]), np.var(mfccs[10]),
                  np.mean(mfccs[11]), np.var(mfccs[11]),
                  np.mean(mfccs[12]), np.var(mfccs[12]),
                  np.mean(mfccs[13]), np.var(mfccs[13]),
                  np.mean(mfccs[14]), np.var(mfccs[14]),
                  np.mean(mfccs[15]), np.var(mfccs[15]),
                  np.mean(mfccs[16]), np.var(mfccs[16]),
                  np.mean(mfccs[17]), np.var(mfccs[17]),
                  np.mean(mfccs[18]), np.var(mfccs[18]),
                  np.mean(mfccs[19]), np.var(mfccs[19])
                  ])
  
  # Scale the data
  inp = min_max_scaler.transform(inp.reshape(1, -1))

  return inp.reshape(1, -1)

inps = [
        get_input_from_file("blues.00000.wav") #,
        # get_input_from_file("blues.00008.wav"),
        # get_input_from_file("pop.00003.wav"),
        # get_input_from_file("pop.00068.wav"),
        # get_input_from_file("classical.00002.wav"),
        # get_input_from_file("classical.00005.wav"),
        # get_input_from_file("bensound-jazzyfrenchy.wav")
]

keys = [
        "blues" #,
        # "blues",
        # "pop",
        # "pop",
        # "classical",
        # "classical",
        # "jazz"
]

print(inps[0])

for i in range(1, 2):
  knn = KNeighborsClassifier(n_neighbors = i)

  # Fit the classifier to the training data
  knn.fit(X, y)

  print("NNeighbors:", i)

  for i in range(len(inps)):
    print(knn.predict(inps[i]) == keys[i], knn.predict(inps[i]), end="; ")
  print("Score:", knn.score(X, y))
  print("-" * 5)



[[0.3782543  0.69505907 0.29273223 0.08666679 0.26441109 0.02690464
  0.46846162 0.06864024 0.35649157 0.0693542  0.20803009 0.01168946
  0.62858191 0.06432259 0.37555228 0.71236432 0.05641986 0.50756741
  0.05588245 0.4596631  0.05632763 0.62075526 0.06415346 0.43836237
  0.10210417 0.59914571 0.09527982 0.43394867 0.09041462 0.52501901
  0.11520492 0.37951592 0.17907948 0.47920918 0.11764173 0.36567409
  0.08265636 0.43483938 0.08543411 0.39189319 0.05923324 0.50131188
  0.0417313  0.43472914 0.07794124 0.41825892 0.07485294 0.42224629
  0.0661303  0.35222487 0.06092947 0.42753399 0.04560534 0.52770927
  0.05125726]]
NNeighbors: 1
[ True] ['blues']; Score: 0.998998998998999
-----
